In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.9 MB/s eta 0:00:00


In [7]:
from ultralytics import YOLO


model_paths = [
    "/content/3.pt",
    "/content/2.pt",
    "/content/1.pt"
]


image_path = "/content/test1.jpg"

for i, path in enumerate(model_paths, start=1):
    print(f"\n--- Проверка модель {i}: {path} ---")
    model = YOLO(path)
    results = model.predict(
        source=image_path,
        conf=0.2,
        save=True,
        project="/content/results",   # папка куда сохранять
        name=f"model_{i}",            # подпапка для каждой модели
        exist_ok=True                 # чтобы не ругался на существующие папки
    )



--- Проверка модель 1: /content/3.pt ---

image 1/1 /content/test1.jpg: 448x640 4 Scratchs, 150.3ms
Speed: 5.5ms preprocess, 150.3ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /content/results/model_1

--- Проверка модель 2: /content/2.pt ---

image 1/1 /content/test1.jpg: 448x640 4 scracths, 148.9ms
Speed: 4.8ms preprocess, 148.9ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /content/results/model_2

--- Проверка модель 3: /content/1.pt ---

image 1/1 /content/test1.jpg: 448x640 2 scratchs, 163.6ms
Speed: 4.2ms preprocess, 163.6ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /content/results/model_3


In [ ]:
!pip install gradio

In [8]:
import io
from ultralytics import YOLO
from PIL import Image
import os # Import os to get basename of model path

def process_car_images(image_files, car_make, license_plate):
    """
    Processes car images using YOLO models to detect damages and tags.

    Args:
        image_files (list): A list of file paths or file-like objects (e.g., from Gradio).
        car_make (str): The make of the car.
        license_plate (str): The license plate number of the car.

    Returns:
        list: A list of dictionaries, where each dictionary contains the
              original image (as a PIL Image), car make, license plate,
              detected tags, and optionally the detected bounding boxes.
    """
    model_paths = [
        "/content/3.pt", # Example model path for damage detection
        "/content/2.pt", # Example model path for rust detection
        "/content/1.pt"  # Example model path for scratch detection
    ]

    # Load models once
    models = []
    for path in model_paths:
        try:
            model = YOLO(path)
            models.append(model)
            # print(f"Successfully loaded model: {path}") # Keep for debugging if needed
        except Exception as e:
            print(f"Error loading model {path}: {e}") # Print model loading errors
            # If a model fails to load, we can't use it for predictions.
            # The list 'models' will only contain successfully loaded models.


    results_data = []

    for i, image_file in enumerate(image_files):
        try:
            # print(f"Processing image {i+1}...") # Keep for debugging if needed
            # Load the image - handle different input types from Gradio
            image = None
            if isinstance(image_file, Image.Image):
                image = image_file.convert("RGB")
                # print(f"Processing image {i+1} from PIL Image object") # Keep for debugging if needed
            elif isinstance(image_file, (str, bytes)): # Handle file paths or bytes
                 try:
                     if isinstance(image_file, str):
                          image = Image.open(image_file).convert("RGB")
                     elif isinstance(image_file, bytes):
                          image = Image.open(io.BytesIO(image_file)).convert("RGB")
                     # print(f"Loaded image {i+1} from path/bytes") # Keep for debugging if needed
                 except Exception as img_load_e:
                     raise IOError(f"Failed to load image {i+1}: {img_load_e}") from img_load_e
            else:
                raise ValueError(f"Unsupported image file type for image {i+1}: {type(image_file)}")


            detected_tags = set()
            detections = []
            all_boxes = [] # To store bounding boxes for potential visualization

            if image is not None:
                # Process image with each successfully loaded model
                for model in models:
                    model_path = getattr(model, 'ckpt_path', 'unknown_model') # Get model path if available
                    try:
                        # print(f"Running prediction on image {i+1} with model {model_path}...") # Keep for debugging if needed
                        # Perform prediction
                        results = model.predict(
                            source=image,
                            conf=0.2, # Confidence threshold
                            save=False, # Don't save images to disk
                            verbose=False # Suppress verbose output
                        )
                        # print(f"Prediction successful for image {i+1} with model {os.path.basename(model_path)}.") # Keep for debugging if needed


                        # Process results from the current model
                        for r in results:
                            if r.boxes is not None and len(r.boxes) > 0:
                                for box in r.boxes:
                                    class_id = int(box.cls)
                                    class_name = model.names[class_id]
                                    detections.append(class_name)
                                    # Store box coordinates and class name
                                    all_boxes.append({
                                        'xyxy': box.xyxy[0].tolist(), # Bounding box coordinates [x1, y1, x2, y2]
                                        'class_name': class_name,
                                        'confidence': float(box.conf)
                                    })
                                # print(f"Detected {len(r.boxes)} objects for image {i+1} with model {os.path.basename(model_path)}. Detections: {detections}") # Keep for debugging if needed
                            # else:
                                # print(f"No objects detected for image {i+1} with model {os.path.basename(model_path)}.") # Keep for debugging if needed


                    except Exception as model_e:
                        print(f"Error during prediction for image {i+1} with model {os.path.basename(model_path)}: {model_e}") # Print prediction errors
                        # Add an error tag for the specific model that failed prediction
                        detected_tags.add(f"Ошибка модели {os.path.basename(model_path)}: {model_e}")


            # Determine tags based on detections across all models
            # Filter out error tags before determining main damage tags
            actual_detections = [d for d in detections if not d.startswith('Ошибка модели')]

            if 'rust' in actual_detections or 'dented' in actual_detections or 'broken' in actual_detections: # Adjust based on your model's classes
                 detected_tags.add('битый')
            if 'scratch' in actual_detections: # Adjust based on your model's classes
                detected_tags.add('царапина')
            # Add logic for 'грязный' if you have a model for it or another criteria

            # Ensure original image is included in the result only if processing was mostly successful
            # (i.e., image loaded and models were available)
            result_image = image if image is not None else None

            results_data.append({
                'original_image': result_image, # Store the PIL Image object or None if loading failed
                'car_make': car_make,
                'license_plate': license_plate,
                'tags': list(detected_tags), # Convert set to list
                'detections': actual_detections, # List of all detected class names (excluding errors)
                'boxes': all_boxes, # List of bounding box details
                'error': None # No general processing error if we reached this point
            })
            # print(f"Finished processing image {i+1}. Tags: {detected_tags}") # Keep for debugging if needed


        except Exception as e:
            print(f"General error processing image {i+1}: {e}") # Print any other general processing errors
            # Append an entry indicating processing failed for this image
            results_data.append({
                'original_image': None, # Indicate failure
                'car_make': car_make,
                'license_plate': license_plate,
                'tags': [f'Общая ошибка обработки: {e}'], # Include general error message in tag
                'detections': [],
                'boxes': [],
                'error': str(e) # Store the error message
            })


    # print("\n--- Final results_data ---") # Keep for debugging if needed
    # for i, result in enumerate(results_data): # Keep for debugging if needed
    #     print(f"Image {i+1}:") # Keep for debugging if needed
    #     print(f"  Car Make: {result.get('car_make')}") # Keep for debugging if needed
    #     print(f"  License Plate: {result.get('license_plate')}") # Keep for debugging if needed
    #     print(f"  Tags: {result.get('tags')}") # Keep for debugging if needed
    #     print(f"  Detections: {result.get('detections')}") # Keep for debugging if needed
    #     print(f"  Boxes: {len(result.get('boxes', []))} detected boxes") # Keep for debugging if needed
    #     if result.get('error'): # Keep for debugging if needed
    #         print(f"  Error: {result['error']}") # Keep for debugging if needed
    # print("--------------------------\n") # Keep for debugging if needed


    return results_data

In [10]:
import gradio as gr
import io
from PIL import Image, ImageDraw, ImageFont

# Assume process_car_images function is defined in the previous step

def process_for_gradio(image1, image2, image3, image4, image5, car_make, license_plate):
    """
    Wrapper function to process images for Gradio output.
    Takes up to 5 Gradio Image components and returns a list of annotated images
    and a formatted string of tags.
    """
    # Collect non-None images and ensure they are PIL Image objects if possible
    image_list = []
    for img in [image1, image2, image3, image4, image5]:
        if img is not None:
            if isinstance(img, Image.Image):
                image_list.append(img)
            # Add other type checks if Gradio might pass other formats (e.g., numpy arrays, file paths)
            # For Gradio's Image component with type="pil", it should ideally pass PIL Image.
            else:
                 print(f"Warning: Unexpected image format received by Gradio wrapper: {type(img)}")
                 # Attempt to convert if it's a path or bytes, though type="pil" should handle this
                 try:
                     if isinstance(img, str): # Assuming it might be a file path
                          image_list.append(Image.open(img).convert("RGB"))
                     elif isinstance(img, bytes): # Assuming it might be bytes
                          image_list.append(Image.open(io.BytesIO(img)).convert("RGB"))
                     # Add other conversions if necessary
                     else:
                          print(f"Error: Cannot process image of type {type(img)}")
                          # Optionally add an error entry to results_data here if conversion fails
                 except Exception as e:
                      print(f"Error converting image type: {e}")
                      # Optionally add an error entry to results_data here if conversion fails


    if not image_list:
        # Return placeholders for outputs if no images are uploaded
        # Also return empty string for overall tags
        return None, None, None, None, None, "", "Пожалуйста, загрузите хотя бы одно изображение."

    # Process images using the core logic
    # The process_car_images function is expected to handle a list of PIL Image objects
    try:
        processing_results = process_car_images(image_list, car_make, license_plate)
    except NameError:
        # Handle case where process_car_images is not defined (e.g., running this cell alone)
        return None, None, None, None, None, "", "Ошибка: Функция обработки изображений не определена."
    except Exception as e:
         return None, None, None, None, None, "", f"Общая ошибка обработки: {e}"


    output_images = []
    output_individual_tags = []
    overall_tags_set = set() # Set to store unique overall tags

    # Prepare output for Gradio
    for i, result in enumerate(processing_results):
        # Check if processing was successful for this image
        if result.get('original_image') and not result.get('error'):
            # Annotate the image with bounding boxes and class names
            annotated_image = result['original_image'].copy()
            draw = ImageDraw.Draw(annotated_image)
            font = None # Use default font

            # Try to load a font
            try:
                font_size = max(10, annotated_image.width // 50) # Dynamic font size
                # Use fonts commonly available in Colab
                font_paths = ["DejaVuSans-Bold.ttf", "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf"]
                for font_path in font_paths:
                    try:
                        font = ImageFont.truetype(font_path, font_size)
                        break # Found a font, exit loop
                    except IOError:
                        continue # Try next font

                if font is None:
                     print("Warning: Font file not found, using default PIL font.")
                     font = ImageFont.load_default()


            except Exception as font_e:
                print(f"Error loading font: {font_e}")
                font = ImageFont.load_default() # Fallback to default font


            for box_info in result.get('boxes', []):
                xyxy = box_info['xyxy']
                class_name = box_info['class_name']
                confidence = box_info['confidence']
                label = f"{class_name}: {confidence:.2f}"

                # Draw bounding box
                draw.rectangle(xyxy, outline="red", width=2)

                # Draw label background and text
                try:
                    text_width, text_height = draw.textbbox((0, 0), label, font=font)[2:]
                except AttributeError: # Handle older PIL versions without textbbox
                     text_width, text_height = draw.textsize(label, font=font)


                # Adjust position to stay within image bounds
                text_x = xyxy[0]
                text_y = xyxy[1] - text_height - 5 # Position above the box

                if text_y < 0: # If text is above top edge, put it below
                    text_y = xyxy[1] + 5
                    # If still outside, adjust x to center if possible
                    if text_y + text_height > annotated_image.height:
                         text_x = xyxy[0] + (xyxy[2] - xyxy[0]) / 2 - text_width / 2
                         text_y = xyxy[1] + 5 # Still below the box

                # Ensure text_x is also within bounds
                if text_x < 0: text_x = 0
                if text_x + text_width > annotated_image.width: text_x = annotated_image.width - text_width

                try:
                    draw.rectangle([text_x, text_y, text_x + text_width, text_y + text_height], fill="red")
                    draw.text((text_x, text_y), label, fill="white", font=font)
                except Exception as draw_e:
                    print(f"Error drawing on image: {draw_e}")


            output_images.append(annotated_image)

            # Add individual image tags to the list
            tags = result.get('tags', [])
            if tags:
                output_individual_tags.append(f"Изображение {i+1}: {', '.join(tags)}")
                # Add tags to the overall set (excluding error tags for overall summary)
                overall_tags_set.update([t for t in tags if not t.startswith('Ошибка')])
            else:
                output_individual_tags.append(f"Изображение {i+1}: Повреждения не обнаружены")

        else:
            # If processing failed or no image was returned, append None
            output_images.append(None)
            # Report error for this specific image
            error_msg = result.get('error', 'Неизвестная ошибка')
            output_individual_tags.append(f"Изображение {i+1}: Ошибка обработки - {error_msg}")
            # Add general error tag to overall tags if applicable
            overall_tags_set.add("Общая ошибка обработки")


    # Filter overall tags to include only "битый" and "грязный" and format for display
    filtered_overall_tags = []
    if 'битый' in overall_tags_set:
        filtered_overall_tags.append('БИТЫЙ')
    if 'грязный' in overall_tags_set:
        filtered_overall_tags.append('ГРЯЗНЫЙ')
    # Add other specific tags if needed, e.g., for errors
    if 'Общая ошибка обработки' in overall_tags_set:
         filtered_overall_tags.append('Общая ошибка обработки') # Include general error tag


    overall_tags_string = "Общие теги повреждений: " + (", ".join(filtered_overall_tags) if filtered_overall_tags else "Нет обнаружено")
    individual_tags_string = "\\n".join(output_individual_tags)

    # Combine overall and individual tags in the final output string
    final_tags_output = f"{overall_tags_string}\\n\\nРезультаты по изображениям:\\n{individual_tags_string}"


    # Gradio expects a fixed number of outputs.
    # If we expect up to 5 images, return a list of 5 images and the tags string.
    # Pad the output_images list with None if fewer than 5 images were processed successfully.
    while len(output_images) < 5:
        output_images.append(None)


    # Return the 5 images and the combined tag string
    return output_images[0], output_images[1], output_images[2], output_images[3], output_images[4], final_tags_output


# Define the Gradio interface
# Use multiple Image components for output, up to a reasonable number (e.g., 5)
# The custom CSS is defined in the previous step and will be applied when the interface is launched.
interface = gr.Interface(
    fn=process_for_gradio,
    inputs=[
        gr.Image(type="pil", label="Загрузите фото автомобиля 1"),
        gr.Image(type="pil", label="Загрузите фото автомобиля 2"),
        gr.Image(type="pil", label="Загрузите фото автомобиля 3"),
        gr.Image(type="pil", label="Загрузите фото автомобиля 4"),
        gr.Image(type="pil", label="Загрузите фото автомобиля 5"),
        gr.Textbox(label="Марка автомобиля"),
        gr.Textbox(label="Госномер")
    ],
    outputs=[
        gr.Image(label="Результат обработки 1", interactive=False),
        gr.Image(label="Результат обработки 2", interactive=False),
        gr.Image(label="Результат обработки 3", interactive=False),
        gr.Image(label="Результат обработки 4", interactive=False),
        gr.Image(label="Результат обработки 5", interactive=False),
        gr.Textbox(label="Обнаруженные теги и информация"),
    ],
    title="Приложение для определения повреждений автомобиля (Indrive Style)",
    description="Загрузите фотографии автомобиля, укажите марку и госномер для определения повреждений.",
    css="""
body {
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif, 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol';
    background-color: #f0f2f5; /* Light grey background */
    color: #333; /* Dark grey text */
}
h1, h2, h3, h4, h5, h6 {
    color: #007bff; /* Indrive blue-ish color for headings */
}
.gradio-container {
    box-shadow: 0 4px 8px rgba(0,0,0,0.1); /* Subtle shadow */
    border-radius: 8px;
    padding: 20px;
    background-color: #ffffff; /* White background for the container */
}
/* Style for text inputs */
.gr-textbox textarea {
    border-color: #ced4da;
    border-radius: 4px;
    padding: 10px;
}
/* Style for buttons */
.gr-button {
    background-color: #007bff; /* Indrive blue-ish color */
    color: white;
    border: none;
    border-radius: 4px;
    padding: 10px 20px;
    font-size: 16px;
    cursor: pointer;
    transition: background-color 0.3s ease;
}
.gr-button:hover {
    background-color: #0056b3; /* Darker blue on hover */
}
/* Style for file upload area */
.gr-upload-text {
    color: #007bff; /* Indrive blue-ish color */
}
/* Style for labels */
.gr-label {
    font-weight: bold;
    color: #555;
}
/* Style for output text */
.gr-text output {
    white-space: pre-wrap; /* Preserve line breaks in output */
}
""", # Apply custom CSS directly
    # theme=gr.themes.Soft() # Example of applying a built-in theme
)

# The interface can be launched using interface.launch()
# The launch command is in a separate cell.

In [12]:
interface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://11d68f6cd7628479c8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Приложение для определения повреждений автомобиля (Indrive Style)

Это веб-приложение, созданное с использованием Gradio и YOLO моделей, для загрузки фотографий автомобилей, определения типов повреждений (таких как "битый", "царапина", "грязный") и отображения результатов.

## Описание

Приложение позволяет пользователям загружать несколько фотографий автомобиля и предоставлять основную информацию (марка, госномер). Затем оно использует предобученные YOLO модели для анализа каждого изображения на наличие повреждений. Результаты обработки включают отображение исходного изображения с нанесенными ограничивающими рамками и метками обнаруженных повреждений, а также сводный список обнаруженных тегов повреждений для всего автомобиля.

Интерфейс приложения стилизован для приближения к дизайну в стиле Indrive.

## Требования

* Python 3.7+
* `gradio`
* `ultralytics`
* `Pillow`
* `torch` и `torchvision` (для работы `ultralytics`)

## Установка и Запуск в Google Colab

Для запуска приложения в Google Colab выполните следующие шаги по порядку:

1. **Установка библиотек:** Выполните первую ячейку кода в ноутбуке, чтобы установить библиотеку `ultralytics`.
2. **Загрузка моделей:** Убедитесь, что файлы ваших обученных YOLO моделей (например, `3.pt`, `2.pt`, `1.pt`) загружены в каталог `/content/` в вашей среде выполнения Colab. Вы можете загрузить их, используя значок папки на левой боковой панели Colab, перейдя в каталог `/content/` и нажав кнопку "Загрузить".
3. **Запуск предсказаний (опционально для проверки):** Ячейка с кодом, которая импортирует `YOLO` и выполняет предсказания (`LUNEQNXLdspQ`), демонстрирует, как использовать модели для обработки изображений. Она настроена на использование моделей из `/content/`. Вы можете запустить ее, чтобы проверить, что модели работают правильно.
4. **Установка Gradio:** Выполните ячейку кода для установки библиотеки `gradio`.
5. **Определение функции обработки и интерфейса Gradio:** Выполните ячейку кода, которая определяет функцию `process_car_images` и `process_for_gradio`, а также сам интерфейс Gradio. Обратите внимание, что в функции `process_car_images` пути к моделям указаны как `/content/3.pt`, `/content/2.pt`, `/content/1.pt`. Убедитесь, что имена файлов моделей в `/content/` совпадают с указанными в коде.
6. **Запуск интерфейса Gradio:** Выполните последнюю ячейку кода, чтобы запустить веб-интерфейс Gradio. После запуска появится публичная ссылка, по которой вы сможете получить доступ к приложению.

## Структура проекта

*   `[ваши файлы моделей]`: Файлы обученных YOLO моделей (например, `3.pt`, `2.pt`, `1.pt`), которые должны быть загружены в `/content/`.
*   `[ваш файл изображения]`: Тестовое изображение (например, `test1.jpg`), которое может быть загружено в `/content/` для проверки.
*   Ячейки кода в ноутбуке:
    *   Установка `ultralytics`.
    *   Пример использования моделей для предсказания на тестовом изображении (опционально).
    *   Установка `gradio`.
    *   Определение функций обработки изображений и интерфейса Gradio.
    *   Запуск интерфейса Gradio.

## Использование

1.  Загрузите фотографии автомобиля с помощью элементов "Загрузите фото автомобиля".
2.  Введите марку автомобиля и госномер в соответствующие текстовые поля.
3.  Нажмите кнопку "Submit" (или аналогичную, в зависимости от языка интерфейса Gradio).
4.  В разделе результатов вы увидите обработанные изображения с обнаруженными повреждениями и список обнаруженных тегов.

## Стилизация

Приложение включает базовую стилизацию с использованием CSS, чтобы придать интерфейсу вид, близкий к стилю Indrive. Вы можете модифицировать CSS в коде интерфейса Gradio для дальнейшей настройки.